In [14]:
#!pip install pattern

In [1]:
import pandas as pd
import numpy as np
import scipy
%matplotlib inline

import re, io ,regex
import spacy
import en_core_web_lg
import textacy

In [2]:
data = pd.read_csv("C:/Users/sohan/Downloads/PHD_ML/Data/Train.csv")
data.head()

,Reviewid,Hotelid,userid,Date,reviewtext,Sentiment
0,Review_1,hotel_101,hotel_1608,"Nov 16, 2007",Nice Marriot View of my king bed room,good
1,Review_2,hotel_101,hotel_6939,"Oct 30, 2007","Good hotel, charges for internet access The Ma...",good
2,Review_3,hotel_101,hotel_3976,"Oct 12, 2007",Small but adequate rooms If you have an early ...,good
3,Review_4,hotel_101,hotel_2851,"Aug 31, 2007","Better than average, some noisy rooms I have s...",good
4,Review_5,hotel_101,hotel_7897,"Jul 18, 2007",Ordinary Although it is highly rated in these ...,bad


In [3]:
data.describe()

,Reviewid,Hotelid,userid,Date,reviewtext,Sentiment
count,6287,6287,6287,6287,6287,6287
unique,6287,100,6100,1310,6287,3
top,Review_3326,hotel_188,hotel_4370,"May 29, 2007","'Perplexed, Oblivious, and Clueless' quoted th...",good
freq,1,521,4,21,1,3213


In [4]:
data.columns

Index(['Reviewid', 'Hotelid', 'userid', 'Date', 'reviewtext', 'Sentiment'], dtype='object')

In [5]:
data['Sentiment']=pd.Categorical(data['Sentiment'],['bad','good','excellent'],ordered=True)

In [6]:
reviewtext=data.reviewtext
target=data.Sentiment

In [7]:
print(reviewtext.head())
print(target.head())
print(reviewtext.shape,target.shape)

0          Nice Marriot       View of my king bed room
1    Good hotel, charges for internet access The Ma...
2    Small but adequate rooms If you have an early ...
3    Better than average, some noisy rooms I have s...
4    Ordinary Although it is highly rated in these ...
Name: reviewtext, dtype: object
0    good
1    good
2    good
3    good
4     bad
Name: Sentiment, dtype: category
Categories (3, object): [bad < good < excellent]
(6287,) (6287,)


### Text Preprocessing

__Sanitising text__

#Changing the text into lower case
reviewtext = reviewtext.apply(lambda x:(textacy.preprocess.preprocess_text(x,lowercase=True)))

Replacing all numbers, emailid, currency with str

In [8]:
reviewtext = reviewtext.apply(lambda x:(textacy.preprocess.replace_numbers(x, replace_with=" ")))

In [9]:
reviewtext = reviewtext.apply(lambda x:(textacy.preprocess.replace_emails(x, replace_with=" ")))

In [10]:
reviewtext = reviewtext.apply(lambda x:(textacy.preprocess.replace_currency_symbols(x, replace_with=" ")))

In [9]:
#For email
reviewtext[2520]

"What are your expectations? Your enjoyment at The Mosser depends greatly on your expectation. This is not to mean that it must be lowered to appreciate the stay. Know what The Mosser is, and arrive with the knowledge that it isn't a  -Star modern resort with tile walls and spacious rooms adjoining rooms, nor is it a carbon copy of hotel chains that deliver mundane and mindless comfort. This is a charming little hotel, in every sense of the adjective. The small rooms contain sparten but modern and functional furnitures. The chair in the corner has an artistic swivel table top that you can pull over your lap comfortbly and work on your laptop, using the wireless internet. The lights are modern decor that retains functionality. The beds, although not the kinds you'd find in a mansion, are neat and comfortable. The in-room bathroom is clean and decently lighted, while the hall bathrooms are equally spotless -- the only difference is that they are in the hallway (I had my own in-room bathr

In [10]:
data.reviewtext[548]

"Great Value Just spent two nights at the Pointe Hilton Squaw Peak (Jan 2 and 3), we had arrived in Phoenix early due to medical appointment and tried checking in rather early (9 AM by hotel standards), had no problem. Housekeeping wasn't completely finished doing our suite so we had breakfast at one of their on-site restaurants and toured their facility (they have a great water park and activity area for kids). Our suite was comfortably appointed and unlike many hotels, clean and fresh smelling (you know what I mean). The king size bed was a pleasure to sleep in, the sheets seem to have a 500 thread count (very soft and luxurious). Our evening meal at their Lantana Grill was adequate. I golfed at their sister resort nearby Lookout Mountain Golf Course great course, my wife stayed at the pool area with the kids, they had a blast. The service from the staff was friendly and courteous; overall we felt that our experience was above average for this class of hotel and very good value for t

In [11]:
#Currency and number replaced ""
reviewtext[548]

"Great Value Just spent two nights at the Pointe Hilton Squaw Peak (Jan   and  ), we had arrived in Phoenix early due to medical appointment and tried checking in rather early (  AM by hotel standards), had no problem. Housekeeping wasn't completely finished doing our suite so we had breakfast at one of their on-site restaurants and toured their facility (they have a great water park and activity area for kids). Our suite was comfortably appointed and unlike many hotels, clean and fresh smelling (you know what I mean). The king size bed was a pleasure to sleep in, the sheets seem to have a   thread count (very soft and luxurious). Our evening meal at their Lantana Grill was adequate. I golfed at their sister resort nearby Lookout Mountain Golf Course great course, my wife stayed at the pool area with the kids, they had a blast. The service from the staff was friendly and courteous; overall we felt that our experience was above average for this class of hotel and very good value for the

__Manual cleaning__

replacing string "showreview" and "full" with "" <br>
showReview(NUMBER,'full'); is a noise in 193 rows

In [12]:
[ t for t in reviewtext[24].split() if (t.startswith('showReview'))|(t.endswith(";"))]

['showReview(', "'full');"]

In [13]:
(reviewtext.apply(lambda x:re.findall(r"\b[s][h][o][w][rR]\S+", x))).sort_values(ascending=False)

2325    [showReview(]
3379    [showReview(]
1868    [showReview(]
5987    [showReview(]
5988    [showReview(]
1861    [showReview(]
1860    [showReview(]
1859    [showReview(]
1858    [showReview(]
1857    [showReview(]
1855    [showReview(]
1854    [showReview(]
1364    [showReview(]
1365    [showReview(]
1366    [showReview(]
260     [showReview(]
2848    [showReview(]
3378    [showReview(]
3380    [showReview(]
4088    [showReview(]
3381    [showReview(]
309     [showReview(]
308     [showReview(]
307     [showReview(]
306     [showReview(]
305     [showReview(]
5655    [showReview(]
5654    [showReview(]
5653    [showReview(]
5652    [showReview(]
            ...      
4195               []
4196               []
4197               []
4198               []
4199               []
4179               []
4177               []
4155               []
4165               []
4156               []
4157               []
4158               []
4159               []
4160               []
4161      

In [11]:
reviewtext=reviewtext.apply(lambda x:re.sub(r"\b[s][h][o][w][rR]\S+","", x))# replacing string "showreview" with ""
reviewtext=reviewtext.apply(lambda x:re.sub(r"\b[f][u][l][l][']\S+","", x))# replacing string "full" with ""

In [15]:
reviewtext[24]

"Kind of worn out!    , '"

In [12]:
#Finding url strings
((reviewtext.apply(lambda x:re.findall(r"(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?", x))).sort_values(ascending=False)).head(6)


3481    [(http, www.youtube.com, /watch?v=tXK37DF1-CQb...
2850    [(http, www.kuletos.com, /cafe/cafekuletos.sht...
2327              [(http, virtualbizweb.com, /?p=23Just)]
6286                                                   []
2097                                                   []
2089                                                   []
Name: reviewtext, dtype: object

In [17]:
reviewtext[2327]

"Worst Service Ever I'll start with the positives: this is a great location close to all San Francisco's best attractions. Within walking distance to convenience stores, museums, Fisherman's Wharf and Pier  , cable car stops, and downtown.My problem here was bad service which I complained about all the way to Marriott Corporate. Our entire stay, the front desk was manned maybe once the whole time. Had payment problems (not our problem, and not one we ever experienced at another Marriott) at check-in which were compounded by a rude clerk.Cell phones would not work in building, which was nobody's fault. But when our family tried to contact us by calling the hotel number, though, nobody would answer the phone to patch them through for our entire trip just about! We had a death of a friend while we were staying here... we did not know about it because our family could not get anyone at the hotel to answer the phone!!Had I not been interested in collecting my Marriott Rewards points, we wou

In [13]:

reviewtext=reviewtext.apply(lambda x:re.sub(r"(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?","", x))# replacing url strings with ""

In [19]:
reviewtext[2327]

"Worst Service Ever I'll start with the positives: this is a great location close to all San Francisco's best attractions. Within walking distance to convenience stores, museums, Fisherman's Wharf and Pier  , cable car stops, and downtown.My problem here was bad service which I complained about all the way to Marriott Corporate. Our entire stay, the front desk was manned maybe once the whole time. Had payment problems (not our problem, and not one we ever experienced at another Marriott) at check-in which were compounded by a rude clerk.Cell phones would not work in building, which was nobody's fault. But when our family tried to contact us by calling the hotel number, though, nobody would answer the phone to patch them through for our entire trip just about! We had a death of a friend while we were staying here... we did not know about it because our family could not get anyone at the hotel to answer the phone!!Had I not been interested in collecting my Marriott Rewards points, we wou

In [20]:
(reviewtext.apply(lambda x:re.findall(u'[⺀-⺙⺛-⻳⼀-⿕々〇〡-〩〸-〺〻㐀-䶵一-鿃豈-鶴侮-頻並-龎]', x))).sort_values(ascending=False)

5319                                            [麓, 麓, 麓]
5674                                               [麓, 麓]
2854                                         [麓, 鈥, 檛, 鈥]
5167                                                  [麓]
47                                                    [麓]
3610                                                  [霉]
4749                                                  [鈧]
2217                                                  [鈧]
152                                    [鈥, 鈥, 鈥, 鈥, 鈥, 鈥]
2596                                      [鈥, 鈥, 鈥, 鈥, 鈥]
4388                                [鈥, 鈥, 鈥, 鈥, 檛, 鈥, 鈥]
3928                                [鈥, 鈥, 鈥, 渂, 鈥, 鈥, 檇]
3262        [鈥, 鈥, 鈥, 檝, 鈥, 檛, 鈥, 鈥, 檚, 鈥, 鈥, 鈥, 檛, 鈥, 檒]
4841                       [鈥, 鈥, 鈥, 檛, 鈥, 鈥, 鈥, 鈥, 鈥, 檇]
4267                    [鈥, 鈥, 鈥, 檚, 鈥, 鈥, 檚, 鈥, 檒, 鈥, 檇]
5377                                [鈥, 鈥, 鈥, 檚, 鈥, 檚, 鈥]
4256                                            [鈥, 鈥, 鈥]
5333    [鈥, 鈥,

In [21]:
reviewtext[2267]

'great location, quiet and nice hotel We really liked the location of this hotel 鈥?on North Point Street, near the activity of the marina but not right in the thick of it. It鈥檚 a very quiet hotel. We had neighbors, but I never heard them once. Nice heated pool and hot tub. The rooms are decorated in a blue and brown modern theme, which we liked very much but which wouldn鈥檛 appeal to everyone. Only a few downsides for us, one of which was the bathroom 鈥?it was beautiful, but the sliding doors didn鈥檛 close completely or lock, and the shower was only half enclosed, meaning that you鈥檙e in there and the back half of the shower was open. This led to water getting out and it being a little colder because the steam could escape. Also, the shower head was a 鈥渞ain鈥?type shower head from the ceiling. Very nice, except that you have to reach under it to turn the water on, so you got a cold blast to the head when you reach forward. And one other picky thing 鈥?the ice bucket is so tiny that we could

In [14]:
# replacing chinese characters with ""
reviewtext=reviewtext.apply(lambda x:re.sub(u'[⺀-⺙⺛-⻳⼀-⿕々〇〡-〩〸-〺〻㐀-䶵一-鿃豈-鶴侮-頻並-龎]',"", x))

In [23]:
reviewtext[2267]

'great location, quiet and nice hotel We really liked the location of this hotel ?on North Point Street, near the activity of the marina but not right in the thick of it. It a very quiet hotel. We had neighbors, but I never heard them once. Nice heated pool and hot tub. The rooms are decorated in a blue and brown modern theme, which we liked very much but which wouldn appeal to everyone. Only a few downsides for us, one of which was the bathroom ?it was beautiful, but the sliding doors didn close completely or lock, and the shower was only half enclosed, meaning that youe in there and the back half of the shower was open. This led to water getting out and it being a little colder because the steam could escape. Also, the shower head was a ain?type shower head from the ceiling. Very nice, except that you have to reach under it to turn the water on, so you got a cold blast to the head when you reach forward. And one other picky thing ?the ice bucket is so tiny that we couldn have two can

In [24]:
(reviewtext.apply(lambda x:re.findall("(\.+)$", x))).sort_values(ascending=False)

2072    [................................................
4760                        [...........................]
4000                              [.....................]
2207                                        [...........]
273                                           [.........]
1392                                            [.......]
2104                                            [.......]
1494                                             [......]
223                                               [.....]
90                                                [.....]
359                                               [.....]
4853                                              [.....]
4519                                              [.....]
3267                                              [.....]
3976                                               [....]
4135                                               [....]
1621                                               [....]
1319          

In [15]:
#Replacing strings like "....." with "."
reviewtext=reviewtext.apply(lambda x:re.sub("(\.+)$",".", x))

In [26]:
(reviewtext.apply(lambda x:re.findall("(\.+)$", x))).sort_values(ascending=False)

761     [.]
3353    [.]
927     [.]
926     [.]
3349    [.]
3350    [.]
3351    [.]
3352    [.]
925     [.]
3328    [.]
3355    [.]
3356    [.]
924     [.]
923     [.]
3359    [.]
3360    [.]
3346    [.]
3345    [.]
3344    [.]
928     [.]
3342    [.]
3341    [.]
3340    [.]
3339    [.]
3338    [.]
3337    [.]
3336    [.]
3335    [.]
3334    [.]
929     [.]
       ... 
3113     []
3111     []
3107     []
3106     []
3103     []
3100     []
3094     []
3158     []
3159     []
3160     []
3198     []
3235     []
3230     []
3228     []
3224     []
3218     []
3210     []
3204     []
3201     []
3197     []
3162     []
3196     []
3194     []
3181     []
3179     []
3178     []
3175     []
3173     []
3170     []
0        []
Name: reviewtext, Length: 6287, dtype: object

In [27]:
(reviewtext.apply(lambda x:re.findall("(\w+\s+\s+\w+)$", x))).sort_values(ascending=False)

5253              [stays       Bathroom]
100                   [stay       lobby]
2338                   [stay       View]
1298               [situacion       bao]
4988                      [shop  Subway]
5676                  [room       Hotel]
1509                 [quiet  restaurant]
2263                [property       Gym]
632             [location       outside]
3547           [location       Exterior]
2059                 [hotel       lobby]
3812              [hotel       entrance]
3111                [hotel       Shower]
5953                 [hotel       Hotel]
1247                   [every   minutes]
2547               [confort       Faade]
2777                 [at       Bathroom]
4859                      [about   mins]
4801                    [US       table]
5123               [Stay       entrance]
4157            [Sofitel       bathroom]
3054                    [Room   Bedroom]
2228        [Renovation       Guestroom]
1218                   [Park       Sofa]
2156            

In [16]:
# Replacing strings like double space "    " with " "
reviewtext=reviewtext.apply(lambda x:re.sub("(\w+\s+\s+\w+)$","", x))

In [29]:
(reviewtext.apply(lambda x:re.findall("(\w+\s+\s+\w+)$", x))).sort_values(ascending=False)

6286    []
2087    []
2089    []
2090    []
2091    []
2092    []
2093    []
2094    []
2095    []
2096    []
2097    []
2098    []
2099    []
2100    []
2101    []
2102    []
2103    []
2104    []
2105    []
2106    []
2107    []
2108    []
2109    []
2088    []
2086    []
1964    []
2085    []
2064    []
2065    []
2066    []
        ..
4217    []
4218    []
4219    []
4220    []
4221    []
4200    []
4199    []
4198    []
4186    []
4177    []
4178    []
4179    []
4180    []
4181    []
4182    []
4183    []
4184    []
4185    []
4187    []
4197    []
4188    []
4189    []
4190    []
4191    []
4192    []
4193    []
4194    []
4195    []
4196    []
0       []
Name: reviewtext, Length: 6287, dtype: object

In [31]:
reviewtext = reviewtext.apply(lambda x:(textacy.preprocess.remove_punct(x, marks=';:/)}(@{*#-_~|')))

In [17]:
from spacy.lang.en import English
nlp = spacy.load("en_core_web_lg")

In [17]:
spacy_reviewtext = reviewtext.apply(nlp)
spacy_reviewtext[113]

Great home base for Seattle sighseeing!! My Husband and I spent a long weekend at the Summerfield suites before an Alaska cruise. The hotel was a steal at   a night. (Hotwire) The hotel is with in walking to pikes market and the space needle. We did use the free hotel shuttle to bring us to the waterfront for dinner. The drive staff were very friendly and got a great tip for dinner!! Elliott oyster bar, one of the best meals I have ever had!! The free breakfast was great!! All in All my stay in Seattle at the Summerfield suites was great!! Clean rooms, great views of the space needle and a friendly, helpful staff!! Would stay again in a heartbeat!!

In [33]:
list(spacy_reviewtext[113].sents)

[Great home base for Seattle sighseeing!!,
 My Husband and I spent a long weekend at the Summerfield suites before an Alaska cruise.,
 The hotel was a steal at   a night.  ,
 Hotwire  ,
 The hotel is with in walking to pikes market and the space needle.,
 We did use the free hotel shuttle to bring us to the waterfront for dinner.,
 The drive staff were very friendly and got a great tip for dinner!!,
 Elliott oyster bar, one of the best meals I have ever had!!,
 The free breakfast was great!!,
 All in All my stay in Seattle at the Summerfield suites was great!!,
 Clean rooms, great views of the space needle and a friendly, helpful staff!!,
 Would stay again in a heartbeat!!]

In [34]:
for i, token in enumerate(spacy_reviewtext[12]):
    #print("original:", token.orth, token.orth_)
    print("lowercased:", token.lower, token.lower_)
    print("lemma:", token.lemma, token.lemma_)
    print("shape:", token.shape, token.shape_)
    print("prefix:", token.prefix, token.prefix_)
    print("suffix:", token.suffix, token.suffix_)
    print("log probability:", token.prob)
    print("Brown cluster id:", token.cluster)
    print("----------------------------------------")
    if i > 1:
        break

lowercased: 2010878274846645280 nickel
lemma: 17836911535321982992 Nickel
shape: 16072095006890171862 Xxxxx
prefix: 10850957288794122991 N
suffix: 18031763335440530313 kel
log probability: -14.784602165222168
Brown cluster id: 614
----------------------------------------
lowercased: 2283656566040971221 and
lemma: 2283656566040971221 and
shape: 4088098365541558500 xxx
prefix: 11901859001352538922 a
suffix: 2283656566040971221 and
log probability: -4.113108158111572
Brown cluster id: 20
----------------------------------------
lowercased: 16362895577960388672 dimed
lemma: 11513532041974030651 Dimed
shape: 16072095006890171862 Xxxxx
prefix: 3929483993944067416 D
suffix: 14009161575225144129 med
log probability: -17.894472122192383
Brown cluster id: 0
----------------------------------------


In [18]:
spacy_reviewtext[113]

Great home base for Seattle sighseeing!! My Husband and I spent a long weekend at the Summerfield suites before an Alaska cruise. The hotel was a steal at   a night. (Hotwire) The hotel is with in walking to pikes market and the space needle. We did use the free hotel shuttle to bring us to the waterfront for dinner. The drive staff were very friendly and got a great tip for dinner!! Elliott oyster bar, one of the best meals I have ever had!! The free breakfast was great!! All in All my stay in Seattle at the Summerfield suites was great!! Clean rooms, great views of the space needle and a friendly, helpful staff!! Would stay again in a heartbeat!!

In [36]:
list(spacy_reviewtext[1].sents)

[Good hotel, charges for internet access,
 The Marriott Airport Seattle is a reasonable choice in the Sea Tac area.,
 I paid about    per night.,
 The shuttle is generally quick to and from the airport,
 and I encourage its use rather taxi service which can be very spotty in Seattle.,
 My room was in the second floor but in the back of the hotel such that I had a very long walk no matter which way I took to the room.,
 The interior is nice and check in was very quick.,
 Breakfast was good in the restaurant, if not a bit pricey.,
 Drink,
 service in the bar was fine, no complaints.,
 I can imagine that it must get very noisy in the lobby when the crowd is hanging out in thebar.,
 I guess Marriott has a new program where guests are now charged for internet service.,
 Its    at this hotel, but can be had for free at the business center near the check in counters.,
 There are two internet PCs and one boarding pass access,
 PC.Personally, the ability to sit in my room and check,
 my persona

In [37]:
review_pos_tag=spacy_reviewtext.apply(lambda x:({w.pos: w.pos_ for w in x}))
review_pos_tag.head()

0    {96: 'PROPN', 103: 'SPACE', 85: 'ADP', 90: 'DE...
1    {84: 'ADJ', 92: 'NOUN', 97: 'PUNCT', 85: 'ADP'...
2    {84: 'ADJ', 89: 'CCONJ', 92: 'NOUN', 85: 'ADP'...
3    {84: 'ADJ', 85: 'ADP', 97: 'PUNCT', 90: 'DET',...
4    {84: 'ADJ', 85: 'ADP', 95: 'PRON', 100: 'VERB'...
Name: reviewtext, dtype: object

__Lemmatize__

In [38]:
(spacy_reviewtext.apply(lambda x:({w.lemma: w.lemma_ for w in x})))[1]

{5711639017775284443: 'good',
 10491429878208300068: 'hotel',
 2593208677638477497: ',',
 16743499924604303818: 'charge',
 16037325823156266367: 'for',
 10937452440342651049: 'internet',
 15459493437284569003: 'access',
 7425985699627899538: 'the',
 11090251104613371396: 'Marriott',
 16083597864187794159: 'Airport',
 715181609030382522: 'Seattle',
 10382539506755952630: 'be',
 11901859001352538922: 'a',
 16093343421169014319: 'reasonable',
 14385143700534068980: 'choice',
 3002984154512732771: 'in',
 291779009535289497: 'Sea',
 17848402196536084053: 'Tac',
 5961633188818984298: 'area',
 12646065887601541794: '.',
 561228191312463089: '-PRON-',
 645224182615914541: 'pay',
 942632335873952620: 'about',
 6718839663412986256: '   ',
 799935474406865112: 'per',
 13511804770843510887: 'night',
 8053403189179670421: 'shuttle',
 18390186386827547778: 'generally',
 12442504647632856847: 'quick',
 3791531372978436496: 'to',
 2283656566040971221: 'and',
 7831658034963690409: 'from',
 128975911652

__Stop Words__

In [39]:
#Importing stop words for English.

stopwords_spacy_default= spacy.lang.en.stop_words.STOP_WORDS
print(list(stopwords_spacy_default)[:10])

['your', 'before', 'something', 'even', 'thereupon', 'although', 'say', 'well', 'whence', 'thereafter']


%time spacy_reviewtext_clean = spacy_reviewtext_lemma.apply(lambda x:[token.text for token in x if not(token.is_punct | token.is_space | token.is_stop)])

In [40]:
spacy_reviewtext[1].vector[:10]

array([ 3.0768581e-02,  1.6200192e-01, -1.1659623e-01, -1.0489543e-01,
        1.4206311e-01,  5.0967996e-04,  3.8513049e-02, -1.8854725e-01,
        1.5390434e-02,  2.2249782e+00], dtype=float32)

In [9]:
# Use the punctuations of string module

from spacy.lang.en.stop_words import STOP_WORDS

stopwords = list(STOP_WORDS)

def spacy_tokenizer(sentence):
    mytokens = parser(sentence)
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]
    mytokens = [word.is_digit]
    mytokens = [ word for word in mytokens if word not in stopwords and word not in punctuations ]
    return mytokens

In [8]:
from sklearn.base import TransformerMixin
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from nltk.corpus import stopwords
import string
import re
import spacy
from spacy.lang.en import English
parser = English()

STOPLIST = set(stopwords.words('english') + list(ENGLISH_STOP_WORDS))
SYMBOLS = " ".join(string.punctuation).split(" ") + ["-", "...", "”", "”"]
class CleanTextTransformer(TransformerMixin):
   def transform(self, X, **transform_params):
        return [cleanText(text) for text in X]
   def fit(self, X, y=None, **fit_params):
        return self
def get_params(self, deep=True):
        return {}
    
def cleanText(text):
    text = text.strip().replace("\n", " ").replace("\r", " ")
    text = text.lower()
    return text
def tokenizeText(sample):
    tokens = parser(sample)
    lemmas = []
    for tok in tokens:
        lemmas.append(tok.lemma_.lower().strip() if tok.lemma_ != "-PRON-" else tok.lower_)
    tokens = lemmas
    tokens = [tok for tok in tokens if tok not in STOPLIST]
    tokens = [tok for tok in tokens if tok not in SYMBOLS]
    return tokens

vectorizer.get_feature_names()

def printNMostInformative(vectorizer, clf, N):
    feature_names = vectorizer.get_feature_names()
    coefs_with_fns = sorted(zip(clf.coef_[0], feature_names))
    topClass1 = coefs_with_fns[:N]
    topClass2 = coefs_with_fns[:-(N + 1):-1]
    topClass3 = coefs_with_fns[:-(N + 2):-1]
    print("Class 1 best: ")
    for feat in topClass1:
        print(feat)
    print("Class 2 best: ")
    for feat in topClass2:
        print(feat)
    print("Class 3 best: ")
    for feat in topClass3:
        print(feat)
vectorizer = CountVectorizer(tokenizer=tokenizeText, ngram_range=(1,1))
clf = LogisticRegression(solver='saga')

pipe = Pipeline([('cleanText', CleanTextTransformer()), ('vectorizer', vectorizer), ('clf', clf)])
# data

train1 = X_train#.tolist()
labelsTrain1 = y_train#.tolist()
val1 = X_val#.tolist()
labelsval1 = y_val#.tolist()
# train
pipe.fit(train1, labelsTrain1)
# val
preds = pipe.predict(val1)
print("accuracy:", accuracy_score(labelsval1, preds))
print("Top 10 features used to predict: ")



printNMostInformative(vectorizer, clf, 20)
pipe = Pipeline([('cleanText', CleanTextTransformer()), ('vectorizer', vectorizer)])
transform = pipe.fit_transform(train1, labelsTrain1)
vocab = vectorizer.get_feature_names()
for i in range(len(train1)):
    s = ""
    indexIntoVocab = transform.indices[transform.indptr[i]:transform.indptr[i+1]]
    numOccurences = transform.data[transform.indptr[i]:transform.indptr[i+1]]
    for idx, num in zip(indexIntoVocab, numOccurences):
        s += str((vocab[idx], num))

In [35]:
import string
import re
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from nltk.corpus import stopwords
from spacy.lang.en.stop_words import STOP_WORDS

punctuations = string.punctuation
REPLACE_BY_SPACE_RE = re.compile("[/(){}\[\]\|@,;'~]")
BAD_SYMBOLS_RE = re.compile('[^a-zA-Z #+_]')
STOPWORDS = set(stopwords.words('english')+ list(ENGLISH_STOP_WORDS)+list(STOP_WORDS))

def cleaning_text(text):
    """
        text: a string
        
        return: modified initial string
    """
#text = textacy.preprocess.replace_numbers(text, replace_with=" ")
#text = textacy.preprocess.replace_emails(text, replace_with=" ")
#text = textacy.preprocess.replace_currency_symbols(text, replace_with=" ")
    text = re.sub(r"\b[s][h][o][w][rR]\S+","", text)# replacing string "showreview" with ""
    text = re.sub(r"\b[f][u][l][l][']\S+","", text)# replacing string "full" with ""
    text = re.sub(r"(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?","", text)# replacing url strings  with ""
    text = re.sub(u'[⺀-⺙⺛-⻳⼀-⿕々〇〡-〩〸-〺〻㐀-䶵一-鿃豈-鶴侮-頻並-龎]',"", text)
    text = re.sub("(\w+\s+\s+\w+)$","", text)
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub('', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub(' ', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwords from text
    return text
parser=English()

def spacy_tokenizer(sentence):
  
    mytokens = parser(sentence)
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]
    mytokens = [ word for word in mytokens if word not in punctuations ]
    return mytokens

In [19]:
def spacy_tokenizer(doc):
    return [x.orth_ for x in nlp(doc)]

In [10]:
reviewtext=reviewtext.apply(cleaning_text)

In [13]:
#train_val_split
X = reviewtext
y = target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=3732)

In [44]:
tfidf = TfidfVectorizer(tokenizer=spacy_tokenizer,strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 1), use_idf=True,smooth_idf=True,sublinear_tf=True,
            stop_words = 'english')
# Fit and transform Tf-idf to both training and test sets
tfidf.fit(list(X_train) + list(X_test))
X_train_tfidf =  tfidf.transform(X_train) 
X_test_tfidf = tfidf.transform(X_test)

In [46]:
countvec = CountVectorizer(analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 1), stop_words = 'english', binary=True)
# Fit and transform CountVectorizer to both training and test sets
countvec.fit(list(X_train) + list(X_test))
X_train_countvec =  countvec.transform(X_train) 
X_test_countvec = countvec.transform(X_test)

## Model Building

In [12]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn import preprocessing, decomposition
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
from sklearn.svm import SVC
import xgboost as xgb

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [52]:
#Feeding the data using tfidf
classifier_tf_nb = MultinomialNB()
classifier_tf_nb.fit(X_train_tfidf,y_train)
#Predicting 
y_pred = classifier_tf_nb.predict(X_test_tfidf)
yt_pred = classifier_tf_nb.predict(X_train_tfidf)
#Analyzing
cm = confusion_matrix(y_test,y_pred)
print(f'Confusion Matrix :\n {cm}\n')
print(f'Val Set Accuracy Score :\n {accuracy_score(y_test,y_pred)}\n')
print(f'Train Set Accuracy Score :\n {accuracy_score(y_train,yt_pred)}\n')
print(f'Classification Report :\n {classification_report(y_test,y_pred)}')

Confusion Matrix :
 [[  3   1 212]
 [  0  11 374]
 [  0  10 647]]

Val Set Accuracy Score :
 0.5254372019077902

Train Set Accuracy Score :
 0.5516007158480811

Classification Report :
              precision    recall  f1-score   support

        bad       1.00      0.01      0.03       216
  excellent       0.50      0.03      0.05       385
       good       0.52      0.98      0.68       657

avg / total       0.60      0.53      0.38      1258



In [53]:
#Feeding the data using Countvector
classifier_cv_nb = MultinomialNB()
classifier_cv_nb.fit(X_train_countvec,y_train)
#Predicting 
y_pred = classifier_cv_nb.predict(X_test_countvec)
yt_pred = classifier_cv_nb.predict(X_train_countvec)
#Analyzing
cm = confusion_matrix(y_test,y_pred)
print(f'Confusion Matrix :\n {cm}\n')
print(f'Val Set Accuracy Score :\n {accuracy_score(y_test,y_pred)}\n')
print(f'Train Set Accuracy Score :\n {accuracy_score(y_train,yt_pred)}\n')
print(f'Classification Report :\n {classification_report(y_test,y_pred)}')

Confusion Matrix :
 [[141   2  73]
 [  2 183 200]
 [ 25 113 519]]

Val Set Accuracy Score :
 0.6701112877583466

Train Set Accuracy Score :
 0.8538476834360708

Classification Report :
              precision    recall  f1-score   support

        bad       0.84      0.65      0.73       216
  excellent       0.61      0.48      0.54       385
       good       0.66      0.79      0.72       657

avg / total       0.67      0.67      0.66      1258



In [54]:
#Feeding the data using tfidf
classifier_tv_lr = LogisticRegression(solver='saga',class_weight='balanced')
classifier_tv_lr.fit(X_train_tfidf,y_train)
#Predicting 
y_pred = classifier_tv_lr.predict(X_test_tfidf)
yt_pred = classifier_tv_lr.predict(X_train_tfidf)
#Analyzing
cm = confusion_matrix(y_test,y_pred)
print(f'Confusion Matrix :\n {cm}\n')
print(f'Val Set Accuracy Score :\n {accuracy_score(y_test,y_pred)}\n')
print(f'Train Set Accuracy Score :\n {accuracy_score(y_train,yt_pred)}\n')
print(f'Classification Report :\n {classification_report(y_test,y_pred)}')

Confusion Matrix :
 [[181   5  30]
 [  9 275 101]
 [ 76 176 405]]

Val Set Accuracy Score :
 0.6844197138314785

Train Set Accuracy Score :
 0.8411214953271028

Classification Report :
              precision    recall  f1-score   support

        bad       0.68      0.84      0.75       216
  excellent       0.60      0.71      0.65       385
       good       0.76      0.62      0.68       657

avg / total       0.70      0.68      0.68      1258



In [55]:
#Feeding the data using Countvector
classifier_cv_lr = LogisticRegression(solver='saga',class_weight='balanced')
classifier_cv_lr.fit(X_train_countvec,y_train)
#Predicting 
y_pred = classifier_cv_lr.predict(X_test_countvec)
yt_pred = classifier_cv_lr.predict(X_train_countvec)
#Analyzing
cm = confusion_matrix(y_test,y_pred)
print(f'Confusion Matrix :\n {cm}\n')
print(f'Val Set Accuracy Score :\n {accuracy_score(y_test,y_pred)}\n')
print(f'Train Set Accuracy Score :\n {accuracy_score(y_train,yt_pred)}\n')
print(f'Classification Report :\n {classification_report(y_test,y_pred)}')

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


Confusion Matrix :
 [[166   8  42]
 [  7 245 133]
 [ 48 177 432]]

Val Set Accuracy Score :
 0.6701112877583466

Train Set Accuracy Score :
 0.9554583416186121

Classification Report :
              precision    recall  f1-score   support

        bad       0.75      0.77      0.76       216
  excellent       0.57      0.64      0.60       385
       good       0.71      0.66      0.68       657

avg / total       0.68      0.67      0.67      1258



In [56]:
testdata_review=testdata_review.apply(cleaning_text)

In [68]:
countvec = CountVectorizer(analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 1), stop_words = 'english', binary=True)
# Fit and transform CountVectorizer to both training and test sets
countvec.fit(list(X_train) + list(X_test))

test_countvec = countvec.transform(testdata_review)

In [62]:
tfidf = TfidfVectorizer(tokenizer=spacy_tokenizer,strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1,1), use_idf=True,smooth_idf=True,sublinear_tf=True,
            stop_words = 'english')
# Fit and transform Tf-idf to both training and test sets
tfidf.fit(list(list(X_train) + list(X_test)))
test_tfidf =  tfidf.transform(testdata_review) 

In [66]:
X_train_tfidf.shape

(5029, 16973)

In [67]:
test_tfidf.shape

(3468, 16973)

In [70]:
classifier_cv_nb.predict(test_countvec)

array(['good', 'bad', 'bad', ..., 'good', 'good', 'excellent'],
      dtype='<U9')

In [ ]:
classifier_tv_lr.predict(test_tfidf)

In [73]:
submission=pd.DataFrame(testdata.Reviewid)

In [76]:
submission['Sentiment']=pd.Series(y_pred)

In [77]:
type(submission)

pandas.core.frame.DataFrame

In [78]:
submission

,Reviewid,Sentiment
0,Review_11001,bad
1,Review_11002,bad
2,Review_11003,good
3,Review_11004,excellent
4,Review_11005,good
5,Review_11006,bad
6,Review_11007,good
7,Review_11008,good
8,Review_11009,excellent
9,Review_11010,good


In [51]:
#Building a pipeline
#Naive Bayes
classifier_nb = MultinomialNB()
classifier_cv_nb = MultinomialNB()

#Logistic regression
classifier_lr = Pipeline([('vect',CountVectorizer(tokenizer=spacy_tokenizer, ngram_range=(1,2))),
                          ('tfidf',TfidfVectorizer(ngram_range=(1,2))),('clf',LogisticRegression(solver='saga'))])
classifier_cv_lr = Pipeline([('vect',CountVectorizer(tokenizer=spacy_tokenizer, ngram_range=(1,1))),('clf',LogisticRegression(solver='saga'))])

#Random Forest
classifier_tv_rf = Pipeline([('tfidf',TfidfVectorizer(tokenizer=spacy_tokenizer, ngram_range=(1,1))),('clf',RandomForestClassifier(bootstrap= False, criterion= 'entropy', n_estimators= 100))])
classifier_cv_rf = Pipeline([('vect',CountVectorizer(tokenizer=spacy_tokenizer, ngram_range=(1,1))),('clf',RandomForestClassifier(bootstrap= False, criterion= 'entropy', n_estimators= 100))])

#Linear SVC
classifier_tv_svc = Pipeline([('tfidf',TfidfVectorizer(tokenizer=spacy_tokenizer, ngram_range=(1,1))),
                     ('clf',LinearSVC())])
classifier_cv_svc = Pipeline([('CountVector',CountVectorizer(tokenizer=spacy_tokenizer, ngram_range=(1,1))),
                     ('clf',LinearSVC())])

##K-Nearest Neighbour
classifier_tv_knn = Pipeline([('tfidf',TfidfVectorizer(tokenizer=spacy_tokenizer, ngram_range=(1,1))),
                     ('clf',KNeighborsClassifier())])
classifier_cv_knn = Pipeline([('vect',CountVectorizer(tokenizer=spacy_tokenizer, ngram_range=(1,1))),
                     ('clf',KNeighborsClassifier())])

##XG Boost Neighbour
classifier_tv_xgb = Pipeline([('tfidf',TfidfVectorizer(tokenizer=spacy_tokenizer, ngram_range=(1,1))),
                     ('clf',xgb.XGBClassifier(max_depth=7,min_child_weight=1,learning_rate=0.1,n_estimators=500))])
classifier_cv_xgb = Pipeline([('vect',CountVectorizer(tokenizer=spacy_tokenizer, ngram_range=(1,1))),
                     ('clf',xgb.XGBClassifier(max_depth=7,min_child_weight=1,learning_rate=0.1,n_estimators=500))])


__Naive Bayes using TfidfVectorizer__

In [52]:
#Feeding the data using tfidf
classifier_tv_nb.fit(X_train,y_train)
#Predicting 
y_pred = classifier_tv_nb.predict(X_val)
yt_pred = classifier_tv_nb.predict(X_train)
#Analyzing
cm = confusion_matrix(y_val,y_pred)
print(f'Confusion Matrix :\n {cm}\n')
print(f'Val Set Accuracy Score :\n {accuracy_score(y_val,y_pred)}\n')
print(f'Train Set Accuracy Score :\n {accuracy_score(y_train,yt_pred)}\n')
print(f'Classification Report :\n {classification_report(y_val,y_pred)}')

AttributeError: 'list' object has no attribute 'lower'

__Naive Bayes using CountVectorizer__

In [42]:
#Feeding the data using CountVectorizer
classifier_cv_nb.fit(X_train,y_train)
#Predicting 
y_pred = classifier_cv_nb.predict(X_val)
yt_pred = classifier_cv_nb.predict(X_train)
#Analyzing
cm = confusion_matrix(y_val,y_pred)
print(f'Confusion Matrix :\n {cm}\n')
print(f'Val Set Accuracy Score :\n {accuracy_score(y_val,y_pred)}\n')
print(f'Train Set Accuracy Score :\n {accuracy_score(y_train,yt_pred)}\n')
print(f'Classification Report :\n {classification_report(y_val,y_pred)}')

AttributeError: lower not found

__Logistic Regression TfidfVectorizer__

In [27]:
classifier_tv_lr.fit(X_train,y_train)
#Predicting 
y_pred = classifier_tv_lr.predict(X_val)
yt_pred = classifier_tv_lr.predict(X_train)
#Analyzing
cm = confusion_matrix(y_val,y_pred)
print(f'Confusion Matrix :\n {cm}\n')
print(f'Val Set Accuracy Score :\n {accuracy_score(y_val,y_pred)}\n')
print(f'Train Set Accuracy Score :\n {accuracy_score(y_train,yt_pred)}\n')
print(f'Classification Report :\n {classification_report(y_val,y_pred)}')

AttributeError: lower not found

__Logistic Regression CountVectorizer__

In [28]:
classifier_cv_lr.fit(X_train,y_train)
#Predicting 
y_pred = classifier_cv_lr.predict(X_val)
yt_pred = classifier_cv_lr.predict(X_train)
#Analyzing
cm = confusion_matrix(y_val,y_pred)
print(f'Confusion Matrix :\n {cm}\n')
print(f'Val Set Accuracy Score :\n {accuracy_score(y_val,y_pred)}\n')
print(f'Train Set Accuracy Score :\n {accuracy_score(y_train,yt_pred)}\n')
print(f'Classification Report :\n {classification_report(y_val,y_pred)}')

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


Confusion Matrix :
 [[148  10  58]
 [  7 234 144]
 [ 29 130 498]]

Val Set Accuracy Score :
 0.699523052464229

Train Set Accuracy Score :
 0.7711274607277789

Classification Report :
              precision    recall  f1-score   support

        bad       0.80      0.69      0.74       216
  excellent       0.63      0.61      0.62       385
       good       0.71      0.76      0.73       657

avg / total       0.70      0.70      0.70      1258



__Random Forest TfidfVectorizer__

In [29]:
classifier_tv_rf.fit(X_train,y_train)
y_pred = classifier_tv_rf.predict(X_val)
yt_pred = classifier_tv_rf.predict(X_train)

cm = confusion_matrix(y_val,y_pred)
print(f'Confusion Matrix :\n {cm}\n')
print(f'Val Set Accuracy Score :\n {accuracy_score(y_val,y_pred)}\n')
print(f'Train Set Accuracy Score :\n {accuracy_score(y_train,yt_pred)}\n')
print(f'Classification Report :\n {classification_report(y_val,y_pred)}')

Confusion Matrix :
 [[ 85   3 128]
 [  1 114 270]
 [  8  52 597]]

Val Set Accuracy Score :
 0.6327503974562798

Train Set Accuracy Score :
 0.9990057665539869

Classification Report :
              precision    recall  f1-score   support

        bad       0.90      0.39      0.55       216
  excellent       0.67      0.30      0.41       385
       good       0.60      0.91      0.72       657

avg / total       0.68      0.63      0.60      1258



__Random Forest CountVectorizer__

In [70]:
classifier_cv_rf.fit(X_train,y_train)
y_pred = classifier_cv_rf.predict(X_val)
yt_pred = classifier_cv_rf.predict(X_train)

cm = confusion_matrix(y_val,y_pred)
print(f'Confusion Matrix :\n {cm}\n')
print(f'Val Set Accuracy Score :\n {accuracy_score(y_val,y_pred)}\n')
print(f'Train Set Accuracy Score :\n {accuracy_score(y_train,yt_pred)}\n')
print(f'Classification Report :\n {classification_report(y_val,y_pred)}')

Confusion Matrix :
 [[148   6 192]
 [ 10 190 375]
 [ 28  96 842]]

Test Set Accuracy Score :
 0.6253312135665077

Train Set Accuracy Score :
 0.9995454545454545

Classification Report :
              precision    recall  f1-score   support

        bad       0.80      0.43      0.56       346
  excellent       0.65      0.33      0.44       575
       good       0.60      0.87      0.71       966

avg / total       0.65      0.63      0.60      1887



__Linear SVC TfidfVectorizer__

In [71]:
classifier_tv_svc.fit(X_train,y_train)
y_pred = classifier_tv_svc.predict(X_val)
yt_pred = classifier_tv_svc.predict(X_train)
cm = confusion_matrix(y_val,y_pred)
print(f'Confusion Matrix :\n {cm}\n')
print(f'Val Set Accuracy Score :\n {accuracy_score(y_val,y_pred)}\n')
print(f'Train Set Accuracy Score :\n {accuracy_score(y_train,yt_pred)}\n')
print(f'Classification Report :\n {classification_report(y_val,y_pred)}')

Confusion Matrix :
 [[237  10  99]
 [ 11 305 259]
 [ 60 216 690]]

Test Set Accuracy Score :
 0.652888182299947

Train Set Accuracy Score :
 0.9711363636363637

Classification Report :
              precision    recall  f1-score   support

        bad       0.77      0.68      0.72       346
  excellent       0.57      0.53      0.55       575
       good       0.66      0.71      0.69       966

avg / total       0.65      0.65      0.65      1887



__Linear SVC CountVectorizer__

In [72]:
classifier_cv_svc.fit(X_train,y_train)
y_pred = classifier_cv_svc.predict(X_val)
yt_pred = classifier_cv_svc.predict(X_train)
cm = confusion_matrix(y_val,y_pred)
print(f'Confusion Matrix :\n {cm}\n')
print(f'Val Set Accuracy Score :\n {accuracy_score(y_val,y_pred)}\n')
print(f'Train Set Accuracy Score :\n {accuracy_score(y_train,yt_pred)}\n')
print(f'Classification Report :\n {classification_report(y_val,y_pred)}')

Confusion Matrix :
 [[223  20 103]
 [ 13 304 258]
 [ 70 259 637]]

Test Set Accuracy Score :
 0.6168521462639109

Train Set Accuracy Score :
 0.9915909090909091

Classification Report :
              precision    recall  f1-score   support

        bad       0.73      0.64      0.68       346
  excellent       0.52      0.53      0.53       575
       good       0.64      0.66      0.65       966

avg / total       0.62      0.62      0.62      1887



__KNN TfidfVectorizer__

In [78]:
classifier_tv_knn.fit(X_train,y_train)
y_pred = classifier_tv_knn.predict(X_val)
yt_pred = classifier_tv_knn.predict(X_train)
cm = confusion_matrix(y_val,y_pred)
print(f'Confusion Matrix :\n {cm}\n')
print(f'Val Set Accuracy Score :\n {accuracy_score(y_val,y_pred)}\n')
print(f'Train Set Accuracy Score :\n {accuracy_score(y_train,yt_pred)}\n')
print(f'Classification Report :\n {classification_report(y_val,y_pred)}')

Confusion Matrix :
 [[138  64 144]
 [ 20 297 258]
 [ 65 341 560]]

Test Set Accuracy Score :
 0.5272919978802332

Train Set Accuracy Score :
 0.7181818181818181

Classification Report :
              precision    recall  f1-score   support

        bad       0.62      0.40      0.49       346
  excellent       0.42      0.52      0.47       575
       good       0.58      0.58      0.58       966

avg / total       0.54      0.53      0.53      1887



__KNN CountVectorizer__

In [79]:
classifier_cv_knn.fit(X_train,y_train)
y_pred = classifier_cv_knn.predict(X_val)
yt_pred = classifier_cv_knn.predict(X_train)
cm = confusion_matrix(y_val,y_pred)
print(f'Confusion Matrix :\n {cm}\n')
print(f'Val Set Accuracy Score :\n {accuracy_score(y_val,y_pred)}\n')
print(f'Train Set Accuracy Score :\n {accuracy_score(y_train,yt_pred)}\n')
print(f'Classification Report :\n {classification_report(y_val,y_pred)}')

Confusion Matrix :
 [[ 64 173 109]
 [ 19 336 220]
 [ 49 428 489]]

Test Set Accuracy Score :
 0.47111817700052994

Train Set Accuracy Score :
 0.6486363636363637

Classification Report :
              precision    recall  f1-score   support

        bad       0.48      0.18      0.27       346
  excellent       0.36      0.58      0.44       575
       good       0.60      0.51      0.55       966

avg / total       0.50      0.47      0.47      1887



__XGB TfidfVectorizer__

In [17]:
%time classifier_tv_xgb.fit(X_train,y_train)
y_pred = classifier_tv_xgb.predict(X_val)
yt_pred = classifier_tv_xgb.predict(X_train)
cm = confusion_matrix(y_val,y_pred)
print(f'Confusion Matrix :\n {cm}\n')
print(f'Val Set Accuracy Score :\n {accuracy_score(y_val,y_pred)}\n')
print(f'Train Set Accuracy Score :\n {accuracy_score(y_train,yt_pred)}\n')
print(f'Classification Report :\n {classification_report(y_val,y_pred)}')

Wall time: 3min 49s


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Confusion Matrix :
 [[226   6 114]
 [ 10 283 282]
 [ 37 168 761]]

Test Set Accuracy Score :
 0.6730259671436142

Train Set Accuracy Score :
 0.990909090909091

Classification Report :
              precision    recall  f1-score   support

        bad       0.83      0.65      0.73       346
  excellent       0.62      0.49      0.55       575
       good       0.66      0.79      0.72       966

avg / total       0.68      0.67      0.67      1887



__XGB CountVectorizer__

In [83]:
%time classifier_cv_xgb.fit(X_train,y_train)
y_pred = classifier_cv_xgb.predict(X_val)
yt_pred = classifier_cv_xgb.predict(X_train)
cm = confusion_matrix(y_val,y_pred)
print(f'Confusion Matrix :\n {cm}\n')
print(f'Val Set Accuracy Score :\n {accuracy_score(y_val,y_pred)}\n')
print(f'Train Set Accuracy Score :\n {accuracy_score(y_train,yt_pred)}\n')
print(f'Classification Report :\n {classification_report(y_val,y_pred)}')

Wall time: 2min 40s


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Confusion Matrix :
 [[222   9 115]
 [  9 275 291]
 [ 39 175 752]]

Test Set Accuracy Score :
 0.661897191308956

Train Set Accuracy Score :
 0.9777272727272728

Classification Report :
              precision    recall  f1-score   support

        bad       0.82      0.64      0.72       346
  excellent       0.60      0.48      0.53       575
       good       0.65      0.78      0.71       966

avg / total       0.67      0.66      0.66      1887



In [84]:
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None, n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5)):
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, val_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    val_scores_mean = np.mean(val_scores, axis=1)
    val_scores_std = np.std(val_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, val_scores_mean - val_scores_std,
                     val_scores_mean + val_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, val_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt
X, y = X_train_svd_scale, y_train
title = "Learning Curves (SVM, RBF kernel, $\gamma=0.001$)"
# SVC is more expensive so we do a lower number of CV iterations:
cv = ShuffleSplit(n_splits=10, val_size=0.2, random_state=0)
estimator = SVC(gamma=0.001)
plot_learning_curve(estimator, title, X, y, (0.5, 1.01), cv=cv, n_jobs=4)
plt.show();

NameError: name 'X_train_svd_scale' is not defined

In [ ]:
model = Pipeline([(‘vectorizer’, CountVectorizer()),
 (‘tfidf’, TfidfTransformer()),
 (‘clf’, OneVsRestClassifier(LinearSVC(class_weight=”balanced”)))])
#the class_weight="balanced" option tries to remove the biasedness of model towards majority sample

In [ ]:
#paramater selection
from sklearn.grid_search import GridSearchCV
parameters = {'vectorizer__ngram_range': [(1, 1), (1, 2),(2,2)],
               'tfidf__use_idf': (True, False)}
gs_clf_svm = GridSearchCV(model, parameters, n_jobs=-1)
gs_clf_svm = gs_clf_svm.fit(X, y)
print(gs_clf_svm.best_score_)
print(gs_clf_svm.best_params_)

In [ ]:
#preparing the final pipeline using the selected parameters
model = Pipeline([('vectorizer', CountVectorizer(ngram_range=(1,2))),
    ('tfidf', TfidfTransformer(use_idf=True)),
    ('clf', OneVsRestClassifier(LinearSVC(class_weight="balanced")))])

In [ ]:
#fit model with training data
model.fit(X_train, y_train)
#evaluation on test data
pred = model.predict(X_test)
from sklearn.metrics import confusion_matrix, accuracy_score
confusion_matrix(pred, y_test)

from sklearn.externals import joblib
model = joblib.load('model_question_topic.pkl')

In [ ]:
from sklearn.externals import joblib
model = joblib.load('model_question_topic.pkl')
question = input()
model.predict([question])[0]

In [ ]:

class_names = ['bad','good','excellent']

losses = []
auc = []

for class_name in class_names:
    #call the labels one column at a time so we can run the classifier on them
    train_target = y_train[class_name]
    test_target = y_test[class_name]
    classifier = LogisticRegression(solver='sag', C=10)

    cv_loss = np.mean(cross_val_score(classifier, X_train_word_features, train_target, cv=5, scoring='neg_log_loss'))
    losses.append(cv_loss)
    print('CV Log_loss score for class {} is {}'.format(class_name, cv_loss))

    cv_score = np.mean(cross_val_score(classifier, X_train_word_features, train_target, cv=5, scoring='accuracy'))
    print('CV Accuracy score for class {} is {}'.format(class_name, cv_score))
    
    classifier.fit(X_train_word_features, train_target)
    y_pred = classifier.predict(test_features)
    y_pred_prob = classifier.predict_proba(test_features)[:, 1]
    auc_score = metrics.roc_auc_score(test_target, y_pred_prob)
    auc.append(auc_score)
    print("CV ROC_AUC score {}\n".format(auc_score))
    
    print(confusion_matrix(test_target, y_pred))
    print(classification_report(test_target, y_pred))

print('Total average CV Log_loss score is {}'.format(np.mean(losses)))
print('Total average CV ROC_AUC score is {}'.format(np.mean(auc)))

## Working with Test data

In [25]:
testdata = pd.read_csv("C:/Users/sohan/Downloads/PHD_ML/Data/Test.csv")
testdata.head()

,Reviewid,Hotelid,userid,Date,reviewtext,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 127,Unnamed: 128,Unnamed: 129,Unnamed: 130,Unnamed: 131,Unnamed: 132,Unnamed: 133,Unnamed: 134,Unnamed: 135,Unnamed: 136
0,Review_11001,hotel_101,hotel_2225,"Dec 13, 2008",Just An Average stay This was just an average ...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Review_11002,hotel_101,hotel_5079,"Dec 2, 2008",go elsewhere The place is hugely overpriced an...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Review_11003,hotel_101,hotel_8440,"Nov 18, 2008",I Won't Go Back I stayed at the hotel 11/14/08...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Review_11004,hotel_101,hotel_4592,"Oct 19, 2008",Good weekend stay My wife and I stay here quit...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Review_11005,hotel_101,hotel_5901,"Oct 13, 2008","Great airport stay Lovely indoor pool area, lo...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
testdata.columns

Index(['Reviewid', 'Hotelid', 'userid', 'Date', 'reviewtext', 'Unnamed: 5',
       'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9',
       ...
       'Unnamed: 127', 'Unnamed: 128', 'Unnamed: 129', 'Unnamed: 130',
       'Unnamed: 131', 'Unnamed: 132', 'Unnamed: 133', 'Unnamed: 134',
       'Unnamed: 135', 'Unnamed: 136'],
      dtype='object', length=137)

In [27]:
testdata=testdata[['Reviewid', 'Hotelid', 'userid', 'Date', 'reviewtext']]

In [28]:
testdata.describe()

,Reviewid,Hotelid,userid,Date,reviewtext
count,3468,3468,3468,3468,3468
unique,3468,100,3403,365,3468
top,Review_13513,hotel_188,hotel_3316,"Mar 28, 2008",Heres the facts from our trip... 2 Queen...
freq,1,161,3,23,1


In [29]:
testdata_review=testdata.reviewtext

__Sanitising text__

In [30]:
import string
import re
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from nltk.corpus import stopwords
from spacy.lang.en.stop_words import STOP_WORDS

REPLACE_BY_SPACE_RE = re.compile("[/(){}\[\]\|@,;'~]")
BAD_SYMBOLS_RE = re.compile('[^a-zA-Z #+_]')
STOPWORDS = set(stopwords.words('english')+ list(ENGLISH_STOP_WORDS)+list(STOP_WORDS))

def cleaning_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = textacy.preprocess.replace_numbers(text, replace_with=" ")
    text = textacy.preprocess.replace_emails(text, replace_with=" ")
    text = textacy.preprocess.replace_currency_symbols(text, replace_with=" ")
    text = re.sub(r"\b[s][h][o][w][rR]\S+","", text)# replacing string "showreview" with ""
    text = re.sub(r"\b[f][u][l][l][']\S+","", text)# replacing string "full" with ""
    text = re.sub(r"(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?","", text)# replacing url strings  with ""
    text = re.sub(u'[⺀-⺙⺛-⻳⼀-⿕々〇〡-〩〸-〺〻㐀-䶵一-鿃豈-鶴侮-頻並-龎]',"", text)
    text = re.sub("(\w+\s+\s+\w+)$","", text)
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub('', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub(' ', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwords from text
    return text

def spacy_tokenizer(sentence):
    mytokens = parser(sentence)
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]
    mytokens = [ word for word in mytokens if word not in stopwords]
    return mytokens

In [ ]:
test_pred = classifier_tv_lr.predict(X_val)